In [126]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
import warnings
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import boto3
import io

pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')

In [127]:
s3 = boto3.client('s3')

# Mention the bucket name where raw data is present
bucket='adl-core-sagemaker-studio'

# Mention the correct directory where raw-data is present, if data is in multiple files make sure all files are present in this directory
file_key = 'external/SDoH/SDoH_Geocoding_Input_UA.csv'
#s3://adl-core-sagemaker-studio/external/SDoH/SDoH_Geocoding_Input_UA.csv


In [128]:
#s3://adl-core-sagemaker-studio/external/SDoH/SDoH_Demographics.csv

In [129]:

s3uri = 's3://{}/{}'.format(bucket, file_key)

df = pd.read_csv(s3uri, dtype={'postal_code': 'string'}, on_bad_lines='skip')

# df = pd.read_csv(s3uri)

df.head()

,udp_global_person_identifier,platform_id,client_id,person_internal_id,address_line_1_text,address_line_2_text,county,city,state,postal_code,postal_code_extnsn,country
0,597ceecd69b3123f10d3941b,245,245,240620038,Soleado Koyasudori 405,"Kanagawa-ku, Koyasudori 1-105",NaN,Yokohama,NaN,2210021,NaN,JPN
1,5984fab611a4b97391711d46,245,245,136620070,2-28-33 Mishuku Setagaya-ku,NaN,NaN,Tokyo,NaN,1540005,NaN,JPN
2,597ced6edab54b2d66d0c168,245,245,422620011,2 11 5 6 Jyosuiminami Cho,NaN,NaN,Kodaira Shi Tokyo,NaN,187,NaN,JPN
3,597cef5011a4b96e2b6a6cb9,245,245,144620052,4 F N32 Lane 165,Tun Hua N. Road,NaN,Taipei,NaN,105,NaN,TWN
4,5985027969b312548933e097,245,245,213620001,1 13 633 Kawaracho,Saiwai Ku,NaN,Kawasaki Shi Kanagawa Ken,NaN,<NA>,NaN,JPN


In [130]:
df.columns

Index(['udp_global_person_identifier', 'platform_id', 'client_id',
       'person_internal_id', 'address_line_1_text', 'address_line_2_text',
       'county', 'city', 'state', 'postal_code', 'postal_code_extnsn',
       'country'],
      dtype='object')

In [131]:
df.isna().sum()

udp_global_person_identifier    15891
platform_id                         0
client_id                           0
person_internal_id                  0
address_line_1_text                11
address_line_2_text             76952
county                          93352
city                               45
state                            9086
postal_code                      1257
postal_code_extnsn              94221
country                           517
dtype: int64

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94221 entries, 0 to 94220
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   udp_global_person_identifier  78330 non-null  object 
 1   platform_id                   94221 non-null  int64  
 2   client_id                     94221 non-null  int64  
 3   person_internal_id            94221 non-null  int64  
 4   address_line_1_text           94210 non-null  object 
 5   address_line_2_text           17269 non-null  object 
 6   county                        869 non-null    object 
 7   city                          94176 non-null  object 
 8   state                         85135 non-null  object 
 9   postal_code                   92964 non-null  string 
 10  postal_code_extnsn            0 non-null      float64
 11  country                       93704 non-null  object 
dtypes: float64(1), int64(3), object(7), string(1)
memory usage: 

In [133]:
df.shape

(94221, 12)

In [134]:
df_2 = df[df['address_line_1_text'].notna() & df['postal_code'].notna()]

In [135]:
df_2.head(5)

,udp_global_person_identifier,platform_id,client_id,person_internal_id,address_line_1_text,address_line_2_text,county,city,state,postal_code,postal_code_extnsn,country
0,597ceecd69b3123f10d3941b,245,245,240620038,Soleado Koyasudori 405,"Kanagawa-ku, Koyasudori 1-105",NaN,Yokohama,NaN,2210021,NaN,JPN
1,5984fab611a4b97391711d46,245,245,136620070,2-28-33 Mishuku Setagaya-ku,NaN,NaN,Tokyo,NaN,1540005,NaN,JPN
2,597ced6edab54b2d66d0c168,245,245,422620011,2 11 5 6 Jyosuiminami Cho,NaN,NaN,Kodaira Shi Tokyo,NaN,187,NaN,JPN
3,597cef5011a4b96e2b6a6cb9,245,245,144620052,4 F N32 Lane 165,Tun Hua N. Road,NaN,Taipei,NaN,105,NaN,TWN
5,59813cc269b3123a5ca06c6c,245,245,117620092,Rua Saint Roman,204,NaN,Rio De Janeiro,NaN,22071060,NaN,BRA


In [136]:
df_2.isna().sum()

udp_global_person_identifier    15552
platform_id                         0
client_id                           0
person_internal_id                  0
address_line_1_text                 0
address_line_2_text             76539
county                          92135
city                               16
state                            7824
postal_code                         0
postal_code_extnsn              92959
country                            22
dtype: int64

In [13]:
# new_df = df.groupby(['address_line_1_text','city','state','postal_code']).sum().reset_index()

In [14]:
# group = new_df[['address_line_1_text','city','state','postal_code']]

In [15]:
# group.shape

In [16]:
# group

In [137]:
df["address_line_1_text"] = df["address_line_1_text"].str.replace("#","")

In [18]:
# group.head()

In [19]:
# group.reset_index(inplace=True)

In [20]:
# group.index = np.arange(1, len(group) + 1)
# group.head()

In [21]:
# group.drop(['index'],axis=1,inplace=True)

In [138]:
import re

def make_zipcode_5digit(zipcode):
    if zipcode.isdigit()==False:
        zipcode = re.sub('\D', '', zipcode)
    if len(zipcode)<5:
        i = 5-len(zipcode)
        zipcode = ('0'*i)+zipcode
    if len(zipcode)>5:
        zipcode = zipcode[:5]
    return zipcode

In [139]:
# make_zipcode_5digit('T1B1H7')

In [140]:
# make_zipcode_5digit('633760000')

In [141]:
df_2['postal_code']=df_2['postal_code'].apply(make_zipcode_5digit)

In [142]:
df_2['address_line_1_text'] = df_2['address_line_1_text'].apply(lambda x: x.strip('.#@!$%^&*'))

In [143]:
df_2.head()

,udp_global_person_identifier,platform_id,client_id,person_internal_id,address_line_1_text,address_line_2_text,county,city,state,postal_code,postal_code_extnsn,country
0,597ceecd69b3123f10d3941b,245,245,240620038,Soleado Koyasudori 405,"Kanagawa-ku, Koyasudori 1-105",NaN,Yokohama,NaN,22100,NaN,JPN
1,5984fab611a4b97391711d46,245,245,136620070,2-28-33 Mishuku Setagaya-ku,NaN,NaN,Tokyo,NaN,15400,NaN,JPN
2,597ced6edab54b2d66d0c168,245,245,422620011,2 11 5 6 Jyosuiminami Cho,NaN,NaN,Kodaira Shi Tokyo,NaN,00187,NaN,JPN
3,597cef5011a4b96e2b6a6cb9,245,245,144620052,4 F N32 Lane 165,Tun Hua N. Road,NaN,Taipei,NaN,00105,NaN,TWN
5,59813cc269b3123a5ca06c6c,245,245,117620092,Rua Saint Roman,204,NaN,Rio De Janeiro,NaN,22071,NaN,BRA


In [144]:
# '_UGPI_' = 'udp_global_person_identifier'
# '_PI_' = 'platform_id'
# '_CI_' = 'client_id'
# '_PID_' = 'person_internal_id'
# created unique_id with above four ids to pass geocoding api as unique_id

In [145]:
df_2['RECORD ID NUMBER'] = '_UGPI_'+df_2['udp_global_person_identifier'].astype(str)+'_PI_'+df_2['platform_id'].astype(str)+'_CI_'+df_2['client_id'].astype(str)+'_PID_'+df_2['person_internal_id'].astype(str)

In [146]:
df_2.head()

,udp_global_person_identifier,platform_id,client_id,person_internal_id,address_line_1_text,address_line_2_text,county,city,state,postal_code,postal_code_extnsn,country,RECORD ID NUMBER
0,597ceecd69b3123f10d3941b,245,245,240620038,Soleado Koyasudori 405,"Kanagawa-ku, Koyasudori 1-105",NaN,Yokohama,NaN,22100,NaN,JPN,_UGPI_597ceecd69b3123f10d3941b_PI_245_CI_245_P...
1,5984fab611a4b97391711d46,245,245,136620070,2-28-33 Mishuku Setagaya-ku,NaN,NaN,Tokyo,NaN,15400,NaN,JPN,_UGPI_5984fab611a4b97391711d46_PI_245_CI_245_P...
2,597ced6edab54b2d66d0c168,245,245,422620011,2 11 5 6 Jyosuiminami Cho,NaN,NaN,Kodaira Shi Tokyo,NaN,00187,NaN,JPN,_UGPI_597ced6edab54b2d66d0c168_PI_245_CI_245_P...
3,597cef5011a4b96e2b6a6cb9,245,245,144620052,4 F N32 Lane 165,Tun Hua N. Road,NaN,Taipei,NaN,00105,NaN,TWN,_UGPI_597cef5011a4b96e2b6a6cb9_PI_245_CI_245_P...
5,59813cc269b3123a5ca06c6c,245,245,117620092,Rua Saint Roman,204,NaN,Rio De Janeiro,NaN,22071,NaN,BRA,_UGPI_59813cc269b3123a5ca06c6c_PI_245_CI_245_P...


In [31]:
#Unique ID, Street address, City, State, ZIP
# just considering address line 1 as address line 2 has lot of nan values 76539 out of 94221

In [147]:
df_3 = df_2[['RECORD ID NUMBER','address_line_1_text', 'city', 'state', 'postal_code']]

In [148]:
df_3.head()

,RECORD ID NUMBER,address_line_1_text,city,state,postal_code
0,_UGPI_597ceecd69b3123f10d3941b_PI_245_CI_245_P...,Soleado Koyasudori 405,Yokohama,NaN,22100
1,_UGPI_5984fab611a4b97391711d46_PI_245_CI_245_P...,2-28-33 Mishuku Setagaya-ku,Tokyo,NaN,15400
2,_UGPI_597ced6edab54b2d66d0c168_PI_245_CI_245_P...,2 11 5 6 Jyosuiminami Cho,Kodaira Shi Tokyo,NaN,00187
3,_UGPI_597cef5011a4b96e2b6a6cb9_PI_245_CI_245_P...,4 F N32 Lane 165,Taipei,NaN,00105
5,_UGPI_59813cc269b3123a5ca06c6c_PI_245_CI_245_P...,Rua Saint Roman,Rio De Janeiro,NaN,22071


In [59]:
# df_3.set_index('unique_id')

In [149]:
df_3.isna().sum()

RECORD ID NUMBER          0
address_line_1_text       0
city                     16
state                  7824
postal_code               0
dtype: int64

In [150]:
df_4 = df_3.dropna(subset=['city','state'], axis=0)

In [151]:
df_4.shape

(85135, 5)

In [152]:
df_4 = df_4.reset_index(drop=True)

In [153]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85135 entries, 0 to 85134
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   RECORD ID NUMBER     85135 non-null  object
 1   address_line_1_text  85135 non-null  object
 2   city                 85135 non-null  object
 3   state                85135 non-null  object
 4   postal_code          85135 non-null  object
dtypes: object(5)
memory usage: 3.2+ MB


In [154]:
# So out of 94221 we got 85135 non nan rows

In [157]:
for row in range(0,len(df_4),10000):
    df_4[row : row+10000].to_excel(f'census_ua/data_{row}_{row+10000}.xlsx',index=False,header=False)

In [158]:
df_4[0:10000].head()

,RECORD ID NUMBER,address_line_1_text,city,state,postal_code
0,_UGPI_58b7bc53f7a4412b0817e5c7_PI_245_CI_245_P...,7069-144th Street,Surrey,BC,00351
1,_UGPI_5e26eb0e69b31214dd575803_PI_245_CI_245_P...,"Flat No 1102, A3, Ivy Building",Thane,MH,40060
2,_UGPI_5c118a2d69b3125eaff5eefa_PI_245_CI_245_P...,7758 Quail Street,Arvada,CO,80005
3,_UGPI_58b7fefd985bac4b586113f1_PI_245_CI_245_P...,558 NH route 101,Temple,NH,03084
4,_UGPI_58b7b467042fec4615e667e8_PI_245_CI_245_P...,621 Boone Trail,Charlottesville,VA,22903
